In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
data_path = r"data\旭诺CTA三号净值20250102.xlsx"
strategy = "量化CTA"
fund_name = "旭诺CTA三号"
benchmark_code = "NH0100.NHF"
benchmark_name = "南华商品指数"
threshold = -0.05

In [3]:
demo = NavResearch(data_path,strategy,fund_name,benchmark_code,benchmark_name,threshold)
data_nav, data_return, data_drawdown = demo.get_data()

In [13]:
import pandas as pd

# 假设get_max_drawdown函数已经定义，并且能正确计算最大回撤
def calculate_annual_metrics(df, fund_name, benchmark_code, benchmark_name):
    years = df['date'].dt.year.unique()
    year_return = pd.DataFrame(columns=["分年度业绩", f"{fund_name}_收益", f"{fund_name}_最大回撤", f"{benchmark_name}_收益", f"{benchmark_name}_最大回撤", "超额收益"])
    nav_year_end = df.sort_values("date").groupby(df["date"].dt.to_period("Y")).tail(1)
    
    for year in years:
        year_df = df[df['date'].dt.year == year]
        fund_start_value = nav_year_end[nav_year_end['date'].dt.year == (year - 1)]['nav_adjusted'].iloc[0] if year > years.min() else year_df['nav_adjusted'].iloc[0]
        fund_end_value = year_df['nav_adjusted'].iloc[-1]
        fund_return = (fund_end_value / fund_start_value - 1)
        fund_max_drawdown = get_max_drawdown(year_df, "nav_adjusted")
        
        benchmark_start_value = nav_year_end[nav_year_end['date'].dt.year == (year - 1)][benchmark_code].iloc[0] if year > years.min() else year_df[benchmark_code].iloc[0]
        benchmark_end_value = year_df[benchmark_code].iloc[-1]
        benchmark_return = (benchmark_end_value / benchmark_start_value - 1)
        benchmark_max_drawdown = get_max_drawdown(year_df, benchmark_code)
        
        excess_return = fund_return - benchmark_return
        new_row = pd.DataFrame([[year, fund_return, fund_max_drawdown, benchmark_return, benchmark_max_drawdown, excess_return]],
                               columns=year_return.columns)
        year_return = pd.concat([year_return, new_row], ignore_index=True)
    
    return year_return

# 使用该函数计算年度指标
year_return_df = calculate_annual_metrics(data_nav, fund_name, benchmark_code, benchmark_name)

# 不转换百分比为字符串，而是使用style来格式化输出
def format_percentages(val):
    return f"{val * 100:.2f}%" if val is not None else "-"
year_return_df =year_return_df.style.format({
    f"{fund_name}_收益": format_percentages,
    f"{fund_name}_最大回撤": format_percentages,  # 假设最大回撤也是一个小数形式的比例
    f"{benchmark_name}_收益": format_percentages,
    f"{benchmark_name}_最大回撤": format_percentages,
    "超额收益": format_percentages,
})
year_return_df

,分年度业绩,旭诺CTA三号_收益,旭诺CTA三号_最大回撤,南华商品指数_收益,南华商品指数_最大回撤,超额收益
0,2019,-5.85%,-6.38%,6.82%,-4.21%,-12.67%
1,2020,81.47%,-15.70%,7.52%,-16.07%,73.95%
2,2021,22.29%,-13.08%,20.94%,-10.76%,1.35%
3,2022,-7.58%,-18.17%,19.66%,-22.72%,-27.24%
4,2023,17.79%,-3.93%,6.21%,-10.98%,11.58%
5,2024,63.81%,-8.42%,-2.08%,-14.10%,65.89%


,分年度业绩,旭诺CTA三号_收益,旭诺CTA三号_最大回撤,南华商品指数_收益,南华商品指数_最大回撤,超额收益
0,2019,-5.85%,-6.38%,6.82%,-4.21%,-12.67%
1,2020,81.47%,-15.70%,7.52%,-16.07%,73.95%
2,2021,22.29%,-13.08%,20.94%,-10.76%,1.35%
3,2022,-7.58%,-18.17%,19.66%,-22.72%,-27.24%
4,2023,17.79%,-3.93%,6.21%,-10.98%,11.58%
5,2024,63.81%,-8.42%,-2.08%,-14.10%,65.89%


In [ ]:
year_return

In [8]:
df_year = (data_nav.sort_values("date").groupby(data_nav["date"].dt.to_period("Y")).tail(1))
first_row_data_nav = data_nav.iloc[0:1]
df_year = pd.concat([first_row_data_nav, df_year], ignore_index=True)
df_year[f"{fund_name}_收益"] = (
    df_year["nav_adjusted"] / df_year["nav_adjusted"].shift(1) - 1
)
df_year[f"{benchmark_name}_收益"] = (
    df_year[benchmark_code] / df_year[benchmark_code].shift(1) - 1
)
df_year["超额收益"] = (
    df_year[f"{fund_name}_收益"] - df_year[f"{benchmark_name}_收益"]
)
df_year["分年度业绩"] = df_year["date"].dt.year
df_year[f"{fund_name}_收益"] = df_year[f"{fund_name}_收益"].apply(
    lambda x: f"{x:.2%}"
)
df_year[f"{benchmark_name}_收益"] = df_year[
    f"{benchmark_name}_收益"
].apply(lambda x: f"{x:.2%}")
df_year["超额收益"] = df_year["超额收益"].apply(lambda x: f"{x:.2%}")
year_return_table = df_year[
    [
        "分年度业绩",
        f"{fund_name}_收益",
        f"{benchmark_name}_收益",
        "超额收益",
    ]
].drop(index=0)